In [1]:
import pandas as pd
from pprint import *

In [2]:
#Load data into dataframes
dv = pd.read_csv('leagueoflegends/deathValues.csv')
#v to add death values in a hierchial scaling 
gv = pd.read_csv('leagueoflegends/goldValues.csv')
#converts gold to double value to parse
ov = pd.read_csv('leagueoflegends/objValues.csv')
bv = pd.read_csv('leagueoflegends/banValues.csv')
#checks bans to see combinations
lol = pd.read_csv('leagueoflegends/_LeagueofLegends.csv')
csvs = [dv, gv, ov, bv, lol]

In [3]:
#for the lol dataframe some lists are in the form of a string, so this is a function to convert a col to a list
def str2list(df, col):
    for i in range(len(df)):
        try:
            df[col][i] = map(float, df[col][i][1:-2].split(","))
        except:
            print i
    return df

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-3-f4232dd17f95>, line 7)

In [ ]:
list_as_str = ['golddiff', 'goldblue', 'bTowers', 'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldblueTop', 'goldblueJungle', ''
              'goldred', 'rTowers', 'rInhibs', 'rDragons', 'rBarons', 'rHeralds']

for col in list_as_str:
    str2list(lol, col)

In [4]:
#because the Kills columns are a list of list in string form we need a seperate function for those
kills = ['bKills', 'rKills']

def str2listoflist(df, col):
    for i in range(len(df)):
        try:
            df[col][i] = df[col][i][1:-2].split(",")[1:-2].split(",")
        except:
            print i
    return df

for col in kills:
    str2listoflist(lol, col)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-4-2827f8bb8f62>, line 9)

In [5]:
d = dv[dv['MatchHistory'] == lol['MatchHistory'][0]]
#d[map(float, d['Time']) < 20]
e = [float(i) for i in d[d['TeamColor'] == 'Red']['Time']]

In [6]:
[len(filter(lambda x: i < x < i + 5, e)) for i in range(0,85,5)]

TypeError: object of type 'filter' has no len()

In [7]:
def count_occ_in_interv(df, index):
    d = df[df['MatchHistory'] == lol['MatchHistory'][index]]
    e = [float(i) for i in d[d['TeamColor'] == 'Red']['Time']]
    f = [float(i) for i in d[d['TeamColor'] == 'Blue']['Time']]
    return ([len(filter(lambda x: i < x < i + 5, e)) for i in range(0,85,5)],
    [len(filter(lambda x: i < x < i + 5, f)) for i in range(0,85,5)])

In [8]:
import numpy as np

roles = ['TopChamp','SupportChamp','MiddleChamp','ADCChamp','JungleChamp']; #Main roles in the current meta, may need to update

champs = pd.read_csv('champs.csv');
champs['wins'] = np.zeros(len(champs['name']));
champs['games'] = np.zeros(len(champs['name']));

games = {'name':champs['name']};
games = pd.DataFrame(data = games)
for name in games['name']:
    games[name] = np.zeros(len(champs['name']));
matchups = {'name':champs['name']};
matchups = pd.DataFrame(data = matchups)
for name in matchups['name']:
    matchups[name] = np.zeros(len(champs['name']));


In [9]:
#get number of wins and games played per champion, and number of wins and games played for each champion matchup

for i in range(len(lol['bResult'])):
    for role in roles: #scraper
        champs.loc[champs['name']==lol['blue' + role][i],'games']+=1;
        champs.loc[champs['name']==lol['red' + role][i],'games']+=1;
        games.loc[games['name']==lol['blue' + role][i],lol['red' + role][i]]+=1;
        games.loc[games['name']==lol['red' + role][i],lol['blue' + role][i]]+=1;
        if lol['bResult'][i] == 1: 
            champs.loc[champs['name']==lol['blue' + role][i],'wins']+=1;
            matchups.loc[matchups['name']==lol['blue' + role][i],lol['red' + role][i]]+=1;
        else:
            champs.loc[champs['name']==lol['red' + role][i],'wins']+=1;
            matchups.loc[matchups['name']==lol['red' + role][i],lol['blue' + role][i]]+=1;

In [10]:
#convert numbers to a percentage
c2 = champs.copy();

g = c2['games'];
g[g==0] = 1;
c2['games'] = g;
c2['wr'] = c2['wins'] / c2['games'];
print(c2)

g2 = games.drop('name',axis=1);
g2[g2 == 0] = 1;

mu = matchups.copy();
for champ in mu:
    if champ != 'name':
        mu[champ] = mu[champ] / g2[champ] - 0.5;
mu = mu.drop('name',axis=1);
mu[games == 0] = 0;
mu['name'] = c2['name'];


             name   id   wins   games        wr
0             Jax   24   37.0    69.0  0.536232
1            Sona   37    9.0    23.0  0.391304
2        Tristana   18   94.0   221.0  0.425339
3           Varus  110  277.0   563.0  0.492007
4           Fiora  114  116.0   229.0  0.506550
5          Singed   27    9.0    15.0  0.600000
6      Tahm Kench  223    0.0     1.0  0.000000
7         Leblanc    7  236.0   467.0  0.505353
8          Thresh  412  457.0   868.0  0.526498
9           Karma   43  404.0   879.0  0.459613
10           Jhin  202  390.0   754.0  0.517241
11         Rumble   68  346.0   663.0  0.521870
12           Udyr   77    6.0    14.0  0.428571
13        Lee Sin   64    0.0     1.0  0.000000
14         Yorick   83    2.0     5.0  0.400000
15           Ornn  516    0.0     1.0  0.000000
16           Kayn  141    0.0     1.0  0.000000
17       Kassadin   38  117.0   214.0  0.546729
18          Sivir   15  626.0  1171.0  0.534586
19    MissFortune   21   62.0   121.0  0

C:\Users\Alec\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
big = pd.read_csv('the biggest baddest csv.csv');

In [12]:
clean = big.drop(['MatchHistory','League','Season', 'Year', 'blueTeamTag', 'rResult', 'redTeamTag'], axis = 1);

In [13]:
clean = clean.drop(['golddiff','goldblue','bKills','bTowers', 'bInhibs', 'bDragons', 'bBarons', 'bHeralds', 'goldred', 'rKills','rTowers','rInhibs','rDragons','rBarons','rHeralds'], axis = 1);

In [14]:
clean = clean.drop(['blueTop','blueJungle','blueMiddle','blueADC','blueSupport','redTop','redJungle','redMiddle','redADC','redSupport'], axis = 1);

In [15]:
clean = clean.drop('Unnamed: 0', axis = 1)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

winner = clean['bResult'];
clean = clean.drop('bResult', axis = 1);

X_train, X_test, y_train, y_test = train_test_split(clean, winner, test_size = 0.33);

In [17]:
import numpy as np

LR = LogisticRegression();

LR.fit(X_train,y_train);

y_predict = LR.predict(X_test)

print((len(y_test) - np.sum(y_test==y_predict)) / len(y_test))

0.0207171314741


In [18]:
pregame = clean.loc[:, :'redSupportChamp'];

In [19]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(pregame, winner, test_size = 0.33);

In [20]:
LR2 = LogisticRegression();

LR2.fit(X_train2,y_train2);

y_predict2 = LR2.predict(X_test2)

print((len(y_test2) - np.sum(y_test2==y_predict2)) / len(y_test2))

0.412749003984


In [21]:
clean_mu = clean.copy();

In [22]:
matchup_names = ['topMu', 'supMu', 'midMu', 'ADCMu','jungMu'];

for i in range(len(matchup_names)):
    clean_mu[matchup_names[i]] = np.zeros(len(clean_mu['redADCChamp']))

In [23]:
mu_matrix = matchups.as_matrix();

print(mu_matrix)

[['Jax' 0.0 0.0 ..., 0.0 0.0 0.0]
 ['Sona' 0.0 0.0 ..., 0.0 0.0 0.0]
 ['Tristana' 0.0 0.0 ..., 0.0 0.0 0.0]
 ..., 
 ['FiddleSticks' 0.0 0.0 ..., 0.0 0.0 0.0]
 ['XinZhao' 0.0 0.0 ..., 0.0 0.0 0.0]
 ['MonkeyKing' 0.0 0.0 ..., 0.0 0.0 0.0]]


In [25]:
champions = matchups['name'].as_matrix();

['Jax' 'Sona' 'Tristana' 'Varus' 'Fiora' 'Singed' 'Tahm Kench' 'Leblanc'
 'Thresh' 'Karma' 'Jhin' 'Rumble' 'Udyr' 'Lee Sin' 'Yorick' 'Ornn' 'Kayn'
 'Kassadin' 'Sivir' 'MissFortune' 'Draven' 'Yasuo' 'Kayle' 'Shaco'
 'Renekton' 'Hecarim' 'Fizz' 'KogMaw' 'Maokai' 'Lissandra' 'Jinx' 'Urgot'
 'Fiddlesticks' 'Galio' 'Pantheon' 'Talon' 'Gangplank' 'Ezreal' 'Gnar'
 'Teemo' 'Annie' 'Mordekaiser' 'Azir' 'Kennen' 'Riven' 'Chogath' 'Aatrox'
 'Poppy' 'Taliyah' 'Illaoi' 'Heimerdinger' 'Alistar' 'Xin Zhao' 'Lucian'
 'Volibear' 'Sejuani' 'Nidalee' 'Garen' 'Leona' 'Zed' 'Blitzcrank' 'Rammus'
 'Velkoz' 'Caitlyn' 'Trundle' 'Kindred' 'Quinn' 'Ekko' 'Nami' 'Swain'
 'Taric' 'Syndra' 'Rakan' 'Skarner' 'Braum' 'Veigar' 'Xerath' 'Corki'
 'Nautilus' 'Ahri' 'Jayce' 'Darius' 'Tryndamere' 'Janna' 'Elise' 'Vayne'
 'Brand' 'Graves' 'Soraka' 'Xayah' 'Karthus' 'Vladimir' 'Zilean' 'Katarina'
 'Shyvana' 'Warwick' 'Ziggs' 'Kled' 'KhaZix' 'Olaf' 'TwistedFate' 'Nunu'
 'Rengar' 'Bard' 'Irelia' 'Ivern' 'Wukong' 'Ashe' 'Kalis

In [46]:
for i in range(len(lol['bResult'])):
    for j in range(len(matchup_names)): 
        clean_mu.set_value(i,matchup_names[j],mu.loc[mu['name']==lol['blue' + roles[j]][i],lol['red' + roles[j]][i]]);

0       -0.029412
1        0.214286
2        0.042857
3        0.057692
4       -0.050000
5        0.042857
6        0.050000
7       -0.064516
8       -0.023810
9        0.202703
10       0.081395
11       0.181818
12       0.014563
13      -0.090361
14       0.158537
15       0.214286
16      -0.057692
17       0.500000
18       0.500000
19      -0.014563
20      -0.023810
21      -0.202703
22      -0.042857
23       0.000000
24       0.029412
25       0.214286
26       0.042857
27       0.000000
28       0.023810
29       0.083333
           ...   
3773    -0.166667
3774    -0.100000
3775     0.000000
3776    -0.500000
3777    -0.031250
3778    -0.040146
3779     0.000000
3780    -0.040146
3781    -0.055556
3782    -0.031250
3783     0.166667
3784     0.500000
3785    -0.031250
3786     0.007937
3787     0.000000
3788    -0.040146
3789    -0.013699
3790     0.013699
3791    -0.031250
3792    -0.250000
3793    -0.100000
3794    -0.031250
3795     0.040146
3796    -0.166667
3797     0

In [50]:
for i in range(len(matchup_names)):
    pregame[matchup_names[i]] = clean_mu[matchup_names[i]];

X_train3, X_test3, y_train3, y_test3 = train_test_split(pregame, winner, test_size = 0.33);

LR3 = LogisticRegression();

LR3.fit(X_train3,y_train3);

y_predict3 = LR3.predict(X_test3)

print((len(y_test3) - np.sum(y_test3==y_predict3)) / len(y_test3))

0.215139442231


In [51]:
print(pregame)

      gamelength  blueTopChamp  blueJungleChamp  blueMiddleChamp  \
0             40      0.503425         0.485884         0.495627   
1             38      0.488987         0.529891         0.495627   
2             40      0.417266         0.529891         0.541126   
3             41      0.503425         0.509158         0.505353   
4             35      0.488987         0.509158         0.505995   
5             24      0.546729         0.529891         0.505353   
6             39      0.503425         0.509158         0.412500   
7             43      0.417266         0.509158         0.531034   
8             41      0.514286         0.485884         0.496350   
9             32      0.503425         0.466667         0.496350   
10            52      0.488987         0.509158         0.531034   
11            46      0.536232         0.509158         0.546729   
12            38      0.503425         0.587302         0.505995   
13            31      0.481081         0.487805 

In [52]:
clean_mu['bResult'] = lol['bResult'];

In [54]:
clean_mu.to_csv('clean and matchups.csv');